# Examples of Shor's Algorithm

In [1]:
import sys
sys.path.append('../../Emulator')
import time
import emulator
import numpy as np
from math import gcd

In [2]:
# Compute the powers of 2 from 0 to 20.
# Observe that 2^14 = 16364 > 8509,
# and that     2^16 = 65536 > 42781.
# These numbers give the value for n in both experiments.
for i in range(20):
    print(str(i) + ': ' + str(2**i))

0: 1
1: 2
2: 4
3: 8
4: 16
5: 32
6: 64
7: 128
8: 256
9: 512
10: 1024
11: 2048
12: 4096
13: 8192
14: 16384
15: 32768
16: 65536
17: 131072
18: 262144
19: 524288


## A Function to Demonstrate Shor's Algorithm.

In [3]:
def demo_shors(a, N, m, n, mults=10, iters=1000, display=False):
    """
    Demonstrate Shor's Algorithm with a given choice of a, N, m, n.
    Computes a^x mod N for all x in the interval [0, 2^m].

    Args:
        a (int): The number to use for factoring.
        N (int): The number to factor.
        m (int): The number of qubits to use on the counting register.
        n (int): The number of quibts to use on the storage register.
                 Must be chosen such that 2^n > N to guarentee success.
        mults (int, optional): The number of multiples to take the estimate 
                               of the period to. Defaults to 10.
        iters (int, optional): The number of times to repeat the process.
                               Defaults to 1000.
        display (bool, optional): Prints a detailed analysis of each iteration
                                  if True. Defaults to False.
    """
    
    # Define variables to hold the time and accuracy.
    times = []
    accuracy = 0
    
    # Repeat the following experiment iters times:
    for i in range(iters):
        
        # Run and time Shor's algorithm to estimate the period of a^x mod N.
        start_time = time.time()
        r = emulator.shors(N, a, m, n)
        times.append(time.time() - start_time)
        
        # Set the initial factor guesses to 1, 1, and success = False.
        guesses = np.array([1, 1])
        success = False
        
        # Compute a few small multiples of r.
        for j in range(mults):
            mult = j + 1
            R = r * mult
            
            # If the R (the power of r) is even:
            if R % 2 == 0:
                
                # Get gcd(a^(R/2)+1, N) and gcd(a^(R/2)-1, N) to guess at factors of N.
                guesses = np.array([gcd(a**(R//2) - 1, N), gcd(a**(R//2) + 1, N)])
                
                # If both guesses are proper factors of N:
                if (guesses[0] != 1 and N % guesses[0] == 0 and
                   guesses[1] != 1 and N % guesses[1] == 0):
                    
                    # Count the trial as a success.
                    accuracy += 1
                    success = True
                    break
        
        # Give a detailed account of each trial if requested.
        if display:
            print('r:' + str(r) + ',\tR:' + str(R) + ',\tmult:' + str(mult) +
                  ', \tGuesses:' + str(guesses) + ',\tPass: ' + str(success))

    # Print the total accuracy and the average time across all trials.
    print('Accuracy:    ', accuracy/iters)
    print('Average time:', sum(times)/iters)

## Experiment 1: N = 8509

In [4]:
# Define N.
N =  67 * 127   # 8509
n = 14  # Again, 2^14 = 16364 > 8509.

# Solve the problem classically to observe the periods of each number.
# Print those with relatively small periods. We will use this to choose our a.
for a in range(2, 67):
    for i in range(1, 500):
        if (a**i)%N == 1:
            print(str(a) + ': ' + str(i))
            break

2: 462
4: 231
5: 462
8: 154
10: 462
16: 231
19: 33
20: 66
22: 99
24: 198
25: 231
27: 462
28: 198
29: 126
30: 126
32: 462
33: 462
37: 9
38: 42
40: 462
47: 231
50: 462
51: 462
52: 198
54: 462
59: 198
61: 462
63: 462
64: 77
66: 42


### Experiment 1.1: a = 38

In [5]:
a = 38  # 38 is the first number with the smallest even period.

In [6]:
# Run Shor's algorithm for various values of m.
# We expect to some improvement when m = n,
# and above 90% accuracy by the time m = 2n (or before).
for m in range(1, 21):
    print('m =', m)
    demo_shors(a, N, m, n)

m = 1
Accuracy:     0.0
Average time: 0.0004838812351226807
m = 2
Accuracy:     0.0
Average time: 0.000512108325958252
m = 3
Accuracy:     0.0
Average time: 0.0007005527019500733
m = 4
Accuracy:     0.0
Average time: 0.0006310617923736572
m = 5
Accuracy:     0.0
Average time: 0.0005677046775817872
m = 6
Accuracy:     0.0
Average time: 0.0006490325927734374
m = 7
Accuracy:     0.0
Average time: 0.0007420494556427002
m = 8
Accuracy:     0.0
Average time: 0.0012619705200195312
m = 9
Accuracy:     0.0
Average time: 0.0014457578659057618
m = 10
Accuracy:     0.0
Average time: 0.0028804733753204346
m = 11
Accuracy:     0.0
Average time: 0.004974963188171387
m = 12
Accuracy:     0.0
Average time: 0.007491432666778565
m = 13
Accuracy:     0.0
Average time: 0.01520790958404541
m = 14
Accuracy:     0.021
Average time: 0.02970949912071228
m = 15
Accuracy:     0.083
Average time: 0.055850502967834476
m = 16
Accuracy:     0.343
Average time: 0.11337353754043579
m = 17
Accuracy:     0.564
Average ti

In [ ]:
# Observe the process in detail for m = 20.
m = 20
demo_shors(a, N, m, n, iters=100, display=True)

r:14,	R:42,	mult:3, 	Guesses:[127  67],	Pass: True
r:2,	R:20,	mult:10, 	Guesses:[1 1],	Pass: False
r:42,	R:42,	mult:1, 	Guesses:[127  67],	Pass: True
r:3,	R:30,	mult:10, 	Guesses:[ 1 67],	Pass: False
r:3,	R:30,	mult:10, 	Guesses:[ 1 67],	Pass: False
r:6,	R:42,	mult:7, 	Guesses:[127  67],	Pass: True
r:21,	R:42,	mult:2, 	Guesses:[127  67],	Pass: True
r:42,	R:42,	mult:1, 	Guesses:[127  67],	Pass: True
r:1,	R:10,	mult:10, 	Guesses:[1 1],	Pass: False
r:42,	R:42,	mult:1, 	Guesses:[127  67],	Pass: True
r:3,	R:30,	mult:10, 	Guesses:[ 1 67],	Pass: False
r:14,	R:42,	mult:3, 	Guesses:[127  67],	Pass: True
r:6,	R:42,	mult:7, 	Guesses:[127  67],	Pass: True
r:42,	R:42,	mult:1, 	Guesses:[127  67],	Pass: True
r:42,	R:42,	mult:1, 	Guesses:[127  67],	Pass: True
r:7,	R:42,	mult:6, 	Guesses:[127  67],	Pass: True
r:42,	R:42,	mult:1, 	Guesses:[127  67],	Pass: True
r:21,	R:42,	mult:2, 	Guesses:[127  67],	Pass: True
r:21,	R:42,	mult:2, 	Guesses:[127  67],	Pass: True
r:21,	R:42,	mult:2, 	Guesses:[127  67],	Pas

### Experiment 1.2: a = 8 (Larger Even Period)

In [5]:
a = 8

# Run Shor's algorithm for various values of m.
# We expect to some improvement when m = n,
# and above 90% accuracy by the time m = 2n (or before).
for m in range(1, 21):
    print('m =', m)
    demo_shors(a, N, m, n)

m = 1
Accuracy:     0.0
Average time: 0.0003392627239227295
m = 2
Accuracy:     0.0
Average time: 0.0003022193908691406
m = 3
Accuracy:     0.0
Average time: 0.00029615592956542966
m = 4
Accuracy:     0.0
Average time: 0.0003397047519683838
m = 5
Accuracy:     0.0
Average time: 0.000330319881439209
m = 6
Accuracy:     0.0
Average time: 0.00035977792739868165
m = 7
Accuracy:     0.0
Average time: 0.00045978307723999024
m = 8
Accuracy:     0.0
Average time: 0.0005767593383789062
m = 9
Accuracy:     0.0
Average time: 0.0008799705505371094
m = 10
Accuracy:     0.0
Average time: 0.001509052038192749
m = 11
Accuracy:     0.0
Average time: 0.0027433135509490965
m = 12
Accuracy:     0.0
Average time: 0.005767303943634033
m = 13
Accuracy:     0.0
Average time: 0.015495291233062744
m = 14
Accuracy:     0.0
Average time: 0.026674251556396484
m = 15
Accuracy:     0.017
Average time: 0.057701557874679565
m = 16
Accuracy:     0.074
Average time: 0.09324052095413209
m = 17
Accuracy:     0.148
Average

In [6]:
# Observe the process in detail for m = 20.
m = 20
demo_shors(a, N, m, n, iters=100, display=True)

r:77,	R:154,	mult:2, 	Guesses:[127  67],	Pass: True
r:11,	R:110,	mult:10, 	Guesses:[ 1 67],	Pass: False
r:154,	R:154,	mult:1, 	Guesses:[127  67],	Pass: True
r:77,	R:154,	mult:2, 	Guesses:[127  67],	Pass: True
r:154,	R:154,	mult:1, 	Guesses:[127  67],	Pass: True
r:77,	R:154,	mult:2, 	Guesses:[127  67],	Pass: True
r:77,	R:154,	mult:2, 	Guesses:[127  67],	Pass: True
r:77,	R:154,	mult:2, 	Guesses:[127  67],	Pass: True
r:77,	R:154,	mult:2, 	Guesses:[127  67],	Pass: True
r:77,	R:154,	mult:2, 	Guesses:[127  67],	Pass: True
r:7769,	R:77690,	mult:10, 	Guesses:[1 1],	Pass: False
r:154,	R:154,	mult:1, 	Guesses:[127  67],	Pass: True
r:77,	R:154,	mult:2, 	Guesses:[127  67],	Pass: True
r:8441,	R:84410,	mult:10, 	Guesses:[1 1],	Pass: False
r:5112,	R:51120,	mult:10, 	Guesses:[1 1],	Pass: False
r:154,	R:154,	mult:1, 	Guesses:[127  67],	Pass: True
r:11,	R:110,	mult:10, 	Guesses:[ 1 67],	Pass: False
r:7,	R:70,	mult:10, 	Guesses:[127   1],	Pass: False
r:1,	R:10,	mult:10, 	Guesses:[1 1],	Pass: False
r:154,

### Experiment 1.3: a = 20 (Even Period Fail)

In [5]:
a = 20

# Run Shor's algorithm for various values of m.
# We expect to some improvement when m = n,
# and above 90% accuracy by the time m = 2n (or before).
for m in range(1, 21):
    print('m =', m)
    demo_shors(a, N, m, n)

m = 1
Accuracy:     0.0
Average time: 0.0005543122291564942
m = 2
Accuracy:     0.0
Average time: 0.00047538471221923826
m = 3
Accuracy:     0.0
Average time: 0.0007086589336395264
m = 4
Accuracy:     0.0
Average time: 0.0006942176818847657
m = 5
Accuracy:     0.0
Average time: 0.0006021716594696044
m = 6
Accuracy:     0.0
Average time: 0.0005529918670654297
m = 7
Accuracy:     0.0
Average time: 0.000607128381729126
m = 8
Accuracy:     0.0
Average time: 0.000952221155166626
m = 9
Accuracy:     0.0
Average time: 0.001181752920150757
m = 10
Accuracy:     0.0
Average time: 0.002065148830413818
m = 11
Accuracy:     0.0
Average time: 0.0035307400226593016
m = 12
Accuracy:     0.0
Average time: 0.006759962797164917
m = 13
Accuracy:     0.0
Average time: 0.012737008571624756
m = 14
Accuracy:     0.0
Average time: 0.032685632944107056
m = 15
Accuracy:     0.0
Average time: 0.054108340740203854
m = 16
Accuracy:     0.0
Average time: 0.10966293883323669
m = 17
Accuracy:     0.0
Average time: 0.2

In [6]:
# Observe the process in detail for m = 20.
m = 20
demo_shors(a, N, m, n, iters=100, display=True)

r:6018,	R:60180,	mult:10, 	Guesses:[127   1],	Pass: False
r:11,	R:110,	mult:10, 	Guesses:[1 1],	Pass: False
r:66,	R:660,	mult:10, 	Guesses:[8509    1],	Pass: False
r:6,	R:60,	mult:10, 	Guesses:[127   1],	Pass: False
r:11,	R:110,	mult:10, 	Guesses:[1 1],	Pass: False
r:33,	R:330,	mult:10, 	Guesses:[   1 8509],	Pass: False
r:66,	R:660,	mult:10, 	Guesses:[8509    1],	Pass: False
r:33,	R:330,	mult:10, 	Guesses:[   1 8509],	Pass: False
r:66,	R:660,	mult:10, 	Guesses:[8509    1],	Pass: False
r:33,	R:330,	mult:10, 	Guesses:[   1 8509],	Pass: False
r:33,	R:330,	mult:10, 	Guesses:[   1 8509],	Pass: False
r:2,	R:20,	mult:10, 	Guesses:[1 1],	Pass: False
r:33,	R:330,	mult:10, 	Guesses:[   1 8509],	Pass: False
r:6,	R:60,	mult:10, 	Guesses:[127   1],	Pass: False
r:33,	R:330,	mult:10, 	Guesses:[   1 8509],	Pass: False
r:33,	R:330,	mult:10, 	Guesses:[   1 8509],	Pass: False
r:66,	R:660,	mult:10, 	Guesses:[8509    1],	Pass: False
r:66,	R:660,	mult:10, 	Guesses:[8509    1],	Pass: False
r:66,	R:660,	mult:

### Experiment 1.3: a = 37 (Odd Period)

In [7]:
a = 37

# Run Shor's algorithm for various values of m.
# We expect to some improvement when m = n,
# and above 90% accuracy by the time m = 2n (or before).
for m in range(1, 21):
    print('m =', m)
    demo_shors(a, N, m, n)

m = 1
Accuracy:     0.0
Average time: 0.00044693899154663085
m = 2
Accuracy:     0.0
Average time: 0.00034422492980957034
m = 3
Accuracy:     0.0
Average time: 0.00036219286918640135
m = 4
Accuracy:     0.0
Average time: 0.0003638777732849121
m = 5
Accuracy:     0.0
Average time: 0.0003984875679016113
m = 6
Accuracy:     0.0
Average time: 0.0004348909854888916
m = 7
Accuracy:     0.0
Average time: 0.0006645872592926026
m = 8
Accuracy:     0.0
Average time: 0.0007101418972015381
m = 9
Accuracy:     0.0
Average time: 0.001199077606201172
m = 10
Accuracy:     0.0
Average time: 0.0018110275268554688
m = 11
Accuracy:     0.0
Average time: 0.0033439116477966308
m = 12
Accuracy:     0.0
Average time: 0.006382925748825073
m = 13
Accuracy:     0.0
Average time: 0.012382638216018677
m = 14
Accuracy:     0.0
Average time: 0.02488133978843689
m = 15
Accuracy:     0.0
Average time: 0.049943477153778074
m = 16
Accuracy:     0.0
Average time: 0.10095197439193726
m = 17
Accuracy:     0.0
Average time:

In [8]:
# Observe the process in detail for m = 20.
m = 20
demo_shors(a, N, m, n, iters=100, display=True)

r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:3,	R:30,	mult:10, 	Guesses:[67  1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:3,	R:30,	mult:10, 	Guesses:[67  1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:9,	R:90,	mult:10, 	Guesses:[8509    1],	Pass: False
r:1,	R:10,	mult:10, 	Guesses:[1 1],	

## Experiment 2: N = 42781

In [ ]:
# Define N.
N = 179 * 239   # 42781

# Solve the problem classically to obtain a reasonable choice of a for this example.
for a in range(2, 179):
    for i in range(1, 2000):
        if (a**i)%N == 1:
            print(str(a) + ': ' + str(i))
            break

10: 1246
22: 1513
24: 1246
36: 1513
38: 1246
44: 1246
51: 1513
67: 1513
75: 1513
98: 1246
100: 623
101: 1513
139: 1246
141: 1246
178: 238


In [ ]:
a = 10  # 10 seems an appropriate choice for a.
n = 16  # Again, 2^16 = 65536 > 42781.

In [ ]:
# Run Shor's algorithm for various values of m.
# We expect to some improvement when m = n,
# and above 90% accuracy by the time m = 2n (or before).
for m in range(1, 21):
    print('m =', m)
    demo_shors(a, N, m, n)

m = 1
Accuracy:     0.0
Average time: 0.0012583355903625487
m = 2
Accuracy:     0.0
Average time: 0.0011813406944274902
m = 3
Accuracy:     0.0
Average time: 0.0011326305866241454
m = 4
Accuracy:     0.0
Average time: 0.001135664463043213
m = 5
Accuracy:     0.0
Average time: 0.0010843122005462647
m = 6
Accuracy:     0.0
Average time: 0.0011460769176483154
m = 7
Accuracy:     0.0
Average time: 0.0012548377513885499
m = 8
Accuracy:     0.0
Average time: 0.0014406991004943847
m = 9
Accuracy:     0.0
Average time: 0.0018825197219848632
m = 10
Accuracy:     0.0
Average time: 0.0031221818923950193
m = 11
Accuracy:     0.0
Average time: 0.004562911748886109
m = 12
Accuracy:     0.0
Average time: 0.008286234378814697
m = 13
Accuracy:     0.0
Average time: 0.015478937149047852
m = 14
Accuracy:     0.0
Average time: 0.030209774494171143
m = 15
Accuracy:     0.0
Average time: 0.06098895812034607
m = 16
Accuracy:     0.0
Average time: 0.12366759872436524
m = 17
Accuracy:     0.007
Average time: 0

In [ ]:
# Observe the process in detail for m = 20.
m = 20
demo_shors(a, N, m, n, iters=100, display=True)

r:22687,	R:204183,	j:9,	Guesses:[239   1],	Pass: False
r:26490,	R:238410,	j:9,	Guesses:[1 1],	Pass: False
r:20117,	R:181053,	j:9,	Guesses:[1 1],	Pass: False
r:27005,	R:243045,	j:9,	Guesses:[1 1],	Pass: False
r:40025,	R:360225,	j:9,	Guesses:[1 1],	Pass: False
r:24265,	R:218385,	j:9,	Guesses:[1 1],	Pass: False
r:42699,	R:384291,	j:9,	Guesses:[1 1],	Pass: False
r:28799,	R:259191,	j:9,	Guesses:[1 1],	Pass: False
r:40119,	R:361071,	j:9,	Guesses:[1 1],	Pass: False
r:38717,	R:348453,	j:9,	Guesses:[239   1],	Pass: False
r:30724,	R:276516,	j:9,	Guesses:[1 1],	Pass: False
r:5827,	R:52443,	j:9,	Guesses:[1 1],	Pass: False
r:20117,	R:181053,	j:9,	Guesses:[1 1],	Pass: False
r:23274,	R:209466,	j:9,	Guesses:[1 1],	Pass: False
r:30000,	R:270000,	j:9,	Guesses:[1 1],	Pass: False
r:1487,	R:13383,	j:9,	Guesses:[1 1],	Pass: False
r:27415,	R:246735,	j:9,	Guesses:[1 1],	Pass: False
r:22615,	R:203535,	j:9,	Guesses:[1 1],	Pass: False
r:24821,	R:223389,	j:9,	Guesses:[1 1],	Pass: False
r:36814,	R:331326,	j:9,	Gue